-sandbox
#Project#3: Exploratory Data Analysis using Databricks 
### CS 5165/6065: Introduction to Cloud Computing

####(Total Points: 20)
####Due Date: 04/23/2020 (End of the day)

####Instruction:
##### Perform exploratory data analysis (EDA) to gain insights using Community Edition of Databricks.
### Use "Project#3: Exploratory Data Analysis using Databricks.dbc" file for the assigment
##### “Databricks Overview Labs CS 5165 | 6065 v1_1.dbc” file has been provided for review

#### Project Tasks:

#####1.  Calculate the Total count of Crimes for each of the 6 United States cities listed below using 2016 crime data in dbfs:/mnt/training/crime-data-2016
#####2.  Provide Total count of different Types of Crimes for each of 6 United States cities listed below using crime-data-2016
#####3.  Calculate the total Robbery count for each of the 3 United States cities listed below using crime-data-2016
#####4.  Find the months with the Highest and Lowest Robbery counts for each of the 3 United States cities listed below using crime-data-2016
#####5.  Combine all three cities robberies-per-month views into one and Find the month with the Highest and Lowest combined Robbery counts using crime-data-2016
#####6.  Plot the robberies per month for each of the three cities, producing one Graph using the contents of combinedRobberiesByMonthDF
#####7.  Find the "per capita robbery rates" using ther Hint below, and plot graph as above for the per capita robbery rates per month for each of the three cities, producing one Graph using the contents of robberyRatesByCityDF
#####8.  Find the monthly HOMICIDE counts for each of the 2 United States cities listed below using crime-data-2016, and Combine both cities HOMICIDE-per-month views into one and Find the month with the Highest and Lowest combined Robbery counts using crime-data-2016
#####9. Find the "per capita HOMICIDE rates" using ther Hint in Qn 7, and plot graph as above for the per capita HOMICIDE rates per month for each of the 2 cities, producing one Graph using the contents of HOMICIDERatesByCityDF
#####10. A stretch goal to address a Data Science question on predicting crimes for a city as a time series model. How would you predict future values for monthly Robbery count rate for Log Angeles using the historical values from crime-data-2016.  Data Science is a multi-year discipline.  I am not expecting anything fancy.  There is no wrong answer. I do expect students to explore and give their best shot.


Submission:
1.	Export the completed "Project#3: Exploratory Data Analysis using Databricks.dbc" as DBC Archive file with all the informatation in Blackboard
2.	Your spark code/command and results included

### Project#3: Exploratory Data Analysis using Databricks
Perform exploratory data analysis (EDA) to gain insights from a data lake.

## Instructions

In `dbfs:/mnt/training/crime-data-2016`, there are a number of Parquet files containing 2016 crime data from seven United States cities:

* New York
* Los Angeles
* Chicago
* Philadelphia
* Dallas
* Boston


The data is cleaned up a little, but has not been normalized. Each city reports crime data slightly differently, so
examine the data for each city to determine how to query it properly.

Your job is to use some of this data to gain insights about certain kinds of crimes.

### Getting Started

Run the following cell to configure our "classroom."

In [4]:
%run ./Includes/Classroom-Setup

In [5]:
print("username: " + username)
print("userhome: " + userhome)
from pyspark.sql.functions import lower, col

username: lohanaa@mail.uc.edu
userhome: dbfs:/user/lohanaa@mail.uc.edu

## Question 1:  Calculate the Total count of Crimes for each of the 6 United States cities listed below using 2016 crime data in dbfs:/mnt/training/crime-data-2016:

* New York
* Los Angeles
* Chicago
* Philadelphia
* Dallas
* Boston


**Hint:**

Start by creating DataFrames for Los Angeles, Philadelphia, and Dallas data.

Use `spark.read.parquet` to create named DataFrames for the files you choose. 

To read in the parquet file, use `crimeDataNewYorkDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-New-York-2016.parquet")`


Use the following view names:

| City          | DataFrame Name            | Path to DBFS file
| ------------- | ------------------------- | -----------------
| Los Angeles   | `crimeDataLosAngelesDF`   | `dbfs:/mnt/training/crime-data-2016/Crime-Data-Los-Angeles-2016.parquet`
| Philadelphia  | `crimeDataPhiladelphiaDF` | `dbfs:/mnt/training/crime-data-2016/Crime-Data-Philadelphia-2016.parquet`
| Dallas        | `crimeDataDallasDF`       | `dbfs:/mnt/training/crime-data-2016/Crime-Data-Dallas-2016.parquet`
| etc...        |

**Hint:**  Los Angeles

In [8]:
crimeDataNewYorkDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-New-York-2016.parquet")
crimeDataLosAngelesDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-Los-Angeles-2016.parquet")
crimeDataPhiladelphiaDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-Philadelphia-2016.parquet")
crimeDataDallasDF=spark.read.parquet("dbfs:/mnt/training/crime-data-2016/Crime-Data-Dallas-2016.parquet")
crimeDataChicagoDF=spark.read.parquet("dbfs:/mnt/training/crime-data-2016/Crime-Data-Chicago-2016.parquet")
crimeDataBostonDF=spark.read.parquet("dbfs:/mnt/training/crime-data-2016/Crime-Data-Boston-2016.parquet")

In [9]:
#task1-Number_of_crimes
crimeDataNewYorkDF.count()

Out[8]: 468241

In [10]:
#task1-Number_of_crimes
crimeDataLosAngelesDF.count()

Out[9]: 217945

In [11]:
crimeDataPhiladelphiaDF.count()

Out[10]: 168664

In [12]:
crimeDataDallasDF.count()

Out[11]: 99642

In [13]:
crimeDataChicagoDF.count()

Out[12]: 267872

In [14]:
crimeDataBostonDF.count()

Out[13]: 218610

-sandbox
## Question 2: Provide Total count of different Types of Crimes for each of 6 United States cities listed below using crime-data-2016:
* New York
* Los Angeles
* Chicago
* Philadelphia
* Dallas
* Boston

### Notice in the Dataframes:
* The `crimeDataNewYorkDF` and `crimeDataBostonDF` DataFrames use different names for the columns.
* The data itself is formatted differently and different names are used for similar concepts.

This is common in a Data Lake. Often files are added to a Data Lake by different groups at different times. The advantage of this strategy is that anyone can contribute information to the Data Lake and that Data Lakes scale to store arbitrarily large and diverse data. The tradeoff for this ease in storing data is that it doesn’t have the rigid structure of a traditional relational data model, so the person querying the Data Lake will need to normalize data before extracting useful insights.

## Same Type of Data, Different Structure

Examine crime data to determine how to extract homicide statistics.

Because the data sets are pooled together in a Data Lake, each city may use different field names and values to indicate homicides, dates, etc.

For example:
* Some cities use the value "HOMICIDE", "CRIMINAL HOMICIDE" or "MURDER".
* In the New York data, the column is named `offenseDescription` while in the Boston data, the column is named `OFFENSE_CODE_GROUP`.
* In the New York data, the date of the event is in the `reportDate`, while in the Boston data, there is a single column named `MONTH`.

In [16]:
crimeDataNewYorkDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-New-York-2016.parquet")
crimeDataLosAngelesDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-Los-Angeles-2016.parquet")
crimeDataPhiladelphiaDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-Philadelphia-2016.parquet")
crimeDataDallasDF=spark.read.parquet("dbfs:/mnt/training/crime-data-2016/Crime-Data-Dallas-2016.parquet")
crimeDataChicagoDF=spark.read.parquet("dbfs:/mnt/training/crime-data-2016/Crime-Data-Chicago-2016.parquet")
crimeDataBostonDF=spark.read.parquet("dbfs:/mnt/training/crime-data-2016/Crime-Data-Boston-2016.parquet")

In [17]:
display(crimeDataNewYorkDF.groupby("offenseDescription").count())

offenseDescription,count
ANTICIPATORY OFFENSES,2
NEW YORK CITY HEALTH CODE,4
OTHER OFFENSES RELATED TO THEF,1708
VEHICLE AND TRAFFIC LAWS,6558
KIDNAPPING & RELATED OFFENSES,132
HOMICIDE-NEGLIGENT-VEHICLE,1
OFF. AGNST PUB ORD SENSBLTY &,22003
PETIT LARCENY OF MOTOR VEHICLE,37
FELONY ASSAULT,20668
OFFENSES RELATED TO CHILDREN,107


In [18]:
display(crimeDataLosAngelesDF.groupby("crimeCodeDescription").count())

crimeCodeDescription,count
GRAND THEFT / INSURANCE FRAUD,4
RECKLESS DRIVING,37
DRIVING WITHOUT OWNER CONSENT (DWOC),64
DISCHARGE FIREARMS/SHOTS FIRED,427
PROWLER,64
THROWING OBJECT AT MOVING VEHICLE,171
"BURGLARY FROM VEHICLE, ATTEMPTED",283
BLOCKING DOOR INDUCTION CENTER,2
INDECENT EXPOSURE,378
CRIMINAL HOMICIDE,293


In [19]:
display(crimeDataPhiladelphiaDF.groupby("ucr_general_description").count())

ucr_general_description,count
LIQUOR LAWS,125
CRIMINAL HOMICIDE,271
WEAPONS,1904
OTHER OFFENSES,29592
ARSON,23929
AGGRAVATED ASSAULT,7592
GAMBLING,43
DUI,3770
MOTOR VEHICLE THEFT,9887
FRAUDS,11859


In [20]:
display(crimeDataDallasDF.groupby("typeOfIncident").count())

typeOfIncident,count
FLEEING POLICE OFFICER,7
VIO BOND/PROTECTIVE ORDER,66
"THEFT OF PROP > OR EQUAL $2,500 BUT <$30K -SHOPLIFT(NOT EMP",74
CAPITAL MURDER OF MULTIPLE PERSONS,8
THEFT OF PROP > OR EQUAL $2500 BUT <$30K ENH -SHOPLIFT-(NOT EMP,1
THEFT OF PROP <$100-OTH THAN SHOPLIFT W/PREV CONVIC ENH,1
BURGLARY OF A COIN OPERATED MACHINE,65
THEFT OF PROP > OR EQUAL $30K<$150K SHOPLIFT (NOT BY EMP),2
UNAUTHORIZED REPRODUCTION OF TEXAS TEMP TAG,1
THEFT OF PROP > OR EQUAL $100 BUT <$750- NOT SHOPLIFT,2392


In [21]:
display(crimeDataChicagoDF.groupby("primaryType").count())

primaryType,count
OFFENSE INVOLVING CHILDREN,2265
STALKING,172
PUBLIC PEACE VIOLATION,1608
OBSCENITY,52
NON-CRIMINAL (SUBJECT SPECIFIED),1
ARSON,519
GAMBLING,189
CRIMINAL TRESPASS,6298
ASSAULT,18746
NON - CRIMINAL,5


In [22]:
display(crimeDataBostonDF.groupby("OFFENSE_CODE_GROUP").count())

OFFENSE_CODE_GROUP,count
Larceny,17876
Auto Theft Recovery,686
Firearm Discovery,508
Recovered Stolen Property,968
License Plate Related Incidents,366
License Violation,1152
Motor Vehicle Accident Response,25274
Liquor Violation,706
Assembly or Gathering Violations,698
Property Found,2601


-sandbox
## Question 3: Calculate the total Robbery count for each of the 3 United States cities listed below using crime-data-2016:
* Los Angeles
* Philadelphia
* Dallas

**Hint:**  For each table, examine the data to figure out how to extract _robbery_ statistics.

Each city uses different values to indicate robbery. Commonly used terminology is "larceny", "burglary" or "robbery." These challenges are common in data lakes.  To simplify things, restrict yourself to only the word "robbery" (and not attempted-roberty, larceny, or burglary).

Explore the data for the three cities until you understand how each city records robbery information. If you don't want to worry about upper- or lower-case, 
remember to use the DataFrame `lower()` method to converts column values to lowercase.

Create a DataFrame containing only the robbery-related rows, as shown in the table below.

**Hint:** For each table, focus your efforts on the column listed below.

Focus on the following columns for each table:

| DataFrame Name            | Robbery DataFrame Name  | Column
| ------------------------- | ----------------------- | -------------------------------
| `crimeDataLosAngelesDF`   | `robberyLosAngelesDF`   | `crimeCodeDescription`
| `crimeDataPhiladelphiaDF` | `robberyPhiladelphiaDF` | `ucr_general_description`
| `crimeDataDallasDF`       | `robberyDallasDF`       | `typeOfIncident`

In [24]:
crimeDataLosAngelesDF.withColumn("crimeCodeDescription",lower(col('crimeCodeDescription'))).filter(crimeDataLosAngelesDF.crimeCodeDescription=="ROBBERY").count()

Out[21]: 9048

In [25]:
crimeDataLosAngelesRobberyDF=crimeDataLosAngelesDF.withColumn("crimeCodeDescription",lower(col('crimeCodeDescription'))).filter(crimeDataLosAngelesDF.crimeCodeDescription.like("%ROBBERY%"))
crimeDataLosAngelesRobberyDF.count()

Out[22]: 10275

In [26]:
crimeDataPhiladelphiaDF.withColumn("ucr_general_description",lower(col('ucr_general_description'))).filter(crimeDataPhiladelphiaDF.ucr_general_description=="ROBBERY").count()

Out[23]: 6149

In [27]:
crimeDataPhiladelphiaRobberyDF=crimeDataPhiladelphiaDF.withColumn("ucr_general_description",lower(col('ucr_general_description'))).filter(crimeDataPhiladelphiaDF.ucr_general_description.like("%ROBBERY%"))
crimeDataPhiladelphiaRobberyDF.count()

Out[24]: 6149

In [28]:
crimeDataDallasRobberyDF=crimeDataDallasDF.filter(crimeDataDallasDF.typeOfIncident.like("%ROBBERY%"))
crimeDataDallasRobberyDF.count()

Out[25]: 6852

-sandbox
## Question 4: Find the months with the Highest and Lowest Robbery counts for each of the 3 United States cities listed below using crime-data-2016:
* Los Angeles
* Philadelphia
* Dallas


**Hint:** Now that you have DataFrames of only the robberies in each city, create DataFrames for each city summarizing the number of robberies in each month.

Your DataFrames must contain two columns:
* `month`: The month number (e.g., 1 for January, 2 for February, etc.).
* `robberies`: The total number of robberies in the month.

Use the following DataFrame names and date columns:


| City          | DataFrame Name     | Date Column 
| ------------- | ------------- | -------------
| Los Angeles   | `robberiesByMonthLosAngelesDF` | `timeOccurred`
| Philadelphia  | `robberiesByMonthPhiladelphiaDF` | `dispatch_date_time`
| Dallas        | `robberiesByMonthDallasDF` | `startingDateTime`

For each city, figure out which column contains the date of the incident. Then, extract the month from that date.

In [30]:
from pyspark.sql.functions import month

In [31]:
rob_la=crimeDataLosAngelesRobberyDF.select(month(crimeDataLosAngelesDF.timeOccurred).alias('MONTH'),crimeDataLosAngelesRobberyDF.crimeCodeDescription).groupBy("MONTH").count()
rob_la_max=rob_la.agg({"count":"max"}).collect()[0][0]
rob_la_min=rob_la.agg({"count":"min"}).collect()[0][0]
print("Month_MAX_ROBBERY",rob_la.select(rob_la.MONTH).where(rob_la['count']==rob_la_max).collect()[0][0])
print("Month_MIN_ROBBERY",rob_la.select(rob_la.MONTH).where(rob_la['count']==rob_la_min).collect()[0][0])

Month_MAX_ROBBERY 12
Month_MIN_ROBBERY 2

In [32]:
rob_phil=crimeDataPhiladelphiaRobberyDF.select(month(crimeDataPhiladelphiaDF.dispatch_date_time).alias('MONTH'),crimeDataPhiladelphiaRobberyDF.ucr_general_description).groupBy("MONTH").count()
rob_phil_max=rob_phil.agg({"count":"max"}).collect()[0][0]
rob_phil_min=rob_phil.agg({"count":"min"}).collect()[0][0]
print("Month_MAX_ROBBERY",rob_phil.select(rob_phil.MONTH).where(rob_phil['count']==rob_phil_max).collect()[0][0])
print("Month_MIN_ROBBERY",rob_phil.select(rob_phil.MONTH).where(rob_phil['count']==rob_phil_min).collect()[0][0])

Month_MAX_ROBBERY 10
Month_MIN_ROBBERY 2

In [33]:
rob_dal=crimeDataDallasRobberyDF.select(month(crimeDataDallasDF.startingDateTime).alias('MONTH'),crimeDataDallasRobberyDF.typeOfIncident).groupBy("MONTH").count()
rob_dal_max=rob_dal.agg({"count":"max"}).collect()[0][0]
rob_dal_min=rob_dal.agg({"count":"min"}).collect()[0][0]
print("Month_MAX_ROBBERY",rob_dal.select(rob_dal.MONTH).where(rob_dal['count']==rob_dal_max).collect()[0][0])
print("Month_MIN_ROBBERY",rob_dal.select(rob_dal.MONTH).where(rob_dal['count']==rob_dal_min).collect()[0][0])

Month_MAX_ROBBERY 1
Month_MIN_ROBBERY 3

-sandbox
## Question 5: Combine all three cities robberies-per-month views into one and Find the month with the Highest and Lowest combined Robbery counts using crime-data-2016:
* Los Angeles
* Philadelphia
* Dallas


Create another DataFrame called `combinedRobberiesByMonthDF`, that combines all three robberies-per-month views into one.
In creating this view, add a new column called `city`, that identifies the city associated with each row.
The final view will have the following columns:

* `city`: The name of the city associated with the row. (Use the strings "Los Angeles", "Philadelphia", and "Dallas".)
* `month`: The month number associated with the row.
* `robbery`: The number of robbery in that month (for that city).

**Hint:** You may want to apply the `union()` method in this example to combine the three datasets.

In [35]:
from pyspark.sql.functions import lit

In [36]:
rob_la= rob_la.withColumn("city",lit("Los Angeles"))
rob_phil= rob_phil.withColumn("city",lit("Philadelphia"))
rob_dal= rob_dal.withColumn("city",lit("Dallas"))

In [37]:
combinedRobberiesByMonthDF = rob_la.union(rob_phil).union(rob_dal)
maxValue = combinedRobberiesByMonthDF.agg({"count" : "max"}).collect()[0][0]
minValue = combinedRobberiesByMonthDF.agg({"count" : "min"}).collect()[0][0]
print("HIGEST_COMBINED_ROBBERY",maxValue)
print("LOWEST_COMBINED_ROBBERY",minValue)

HIGEST_COMBINED_ROBBERY 958
LOWEST_COMBINED_ROBBERY 414

-sandbox
## Question 6: Plot the robberies per month for each of the three cities, producing one Graph using the contents of `combinedRobberiesByMonthDF`

Adjust the plot options to configure the plot properly, as shown below:

When you first run the cell, you'll get an HTML table as the result. To configure the plot:

<img src="https://files.training.databricks.com/images/eLearning/capstone-plot-4.png" style="width: 362px; margin: 10px; border: 1px solid #aaaaaa; border-radius: 10px 10px 10px 10px"/>

1. Click the graph button.
2. If the plot doesn't look correct, click the **Plot Options** button.
3. Configure the plot similar to the following example.

;**Hint:** Order your results by `month`, then `city`.

<img src="https://files.training.databricks.com/images/eLearning/capstone-plot-2.png" style="width: 268px; margin: 10px; border: 1px solid #aaaaaa; border-radius: 10px 10px 10px 10px"/>

In [39]:
display(combinedRobberiesByMonthDF)

MONTH,count,city
12,958,Los Angeles
1,835,Los Angeles
6,827,Los Angeles
3,810,Los Angeles
5,888,Los Angeles
9,834,Los Angeles
4,804,Los Angeles
8,850,Los Angeles
7,931,Los Angeles
10,932,Los Angeles


## Question 7: Find the "per capita robbery rates" using ther Hint below, and plot graph as above for the per capita robbery rates per month for each of the three cities, producing one Graph using the contents of `robberyRatesByCityDF`:
* Los Angeles
* Philadelphia
* Dallas

While the above graph is interesting, it's flawed: it's comparing the raw numbers of robberies, not the per capita robbery rates.

The DataFrame (already created) called `cityDataDF` (dbfs:/mnt/training/City-Data.parquet)  contains, among other data, estimated 2016 population values for all United States cities
with populations of at least 100,000. (The data is from [Wikipedia](https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population).)

* Use the population values in that table to normalize the robberies so they represent per-capita values (total robberies divided by population).
* Save your results in a DataFrame called `robberyRatesByCityDF`.
* The robbery rate value must be stored in a new column, `robberyRate`.

Next, graph the results, as above.

In [41]:
cityDataDF = spark.read.parquet("/mnt/training/City-Data.parquet")
cityDataDF = cityDataDF.select(cityDataDF.estPopulation2016,cityDataDF.city).where(cityDataDF.city.isin("Los Angeles","Chicago","Boston","Dallas","New Orleans","New York","Philadelphia"))
cityDataDF.show()

+-----------------+------------+
estPopulation2016| city|
+-----------------+------------+
 8537673| New York|
 3976322| Los Angeles|
 2704958| Chicago|
 1567872|Philadelphia|
 1317929| Dallas|
 673184| Boston|
 391495| New Orleans|
+-----------------+------------+

In [42]:
import pyspark.sql.functions as F
robberyRatesByCityDF = combinedRobberiesByMonthDF.withColumn("robberyRate",F.col("count") / cityDataDF.select(cityDataDF.estPopulation2016).where(cityDataDF.city == F.col("city")).collect()[0][0])
display(robberyRatesByCityDF)

MONTH,count,city,robberyRate
12,958,Los Angeles,1.1220856081042224E-4
1,835,Los Angeles,9.78018249234891E-5
6,827,Los Angeles,9.686480145116825E-5
3,810,Los Angeles,9.487362657248644E-5
5,888,Los Angeles,1.0400960542761477E-4
9,834,Los Angeles,9.7684696989449E-5
4,804,Los Angeles,9.41708589682458E-5
8,850,Los Angeles,9.95587439340907E-5
7,931,Los Angeles,1.0904610659133935E-4
10,932,Los Angeles,1.0916323452537946E-4


-sandbox
## Question 8: Find the monthly HOMICIDE counts for each of the 2 United States cities listed below using crime-data-2016, and Combine both cities HOMICIDE-per-month views into one and Find the month with the Highest and Lowest combined Robbery counts.  See Question 6.
* New York
* Boston

**Hint:**  Same Type of Data, Different Structure
In this section, we examine crime data to determine how to extract homicide statistics.
Each city may use different field names and values to indicate homicides, dates, etc.
For example:
* Some cities use the value "HOMICIDE", "CRIMINAL HOMICIDE" or "MURDER".
* In the New York data, the column is named `offenseDescription` while in the Boston data, the column is named `OFFENSE_CODE_GROUP`.
* In the New York data, the date of the event is in the `reportDate`, while in the Boston data, there is a single column named `MONTH`.

To get started, create a temporary view containing only the homicide-related rows.
At the same time, normalize the data structure of each table so all the columns (and their values) line up with each other.
In the case of New York and Boston, here are the unique characteristics of each data set:

| | Offense-Column        | Offense-Value          | Reported-Column  | Reported-Data Type |
|-|-----------------------|------------------------|-----------------------------------|
| New York | `offenseDescription`  | starts with "murder" or "homicide" | `reportDate`     | `timestamp`    |
| Boston | `OFFENSE_CODE_GROUP`  | "Homicide"             | `MONTH`          | `integer`      |
For the upcoming aggregation, you need to alter the New York data set to include a `month` column which can be computed from the `reportDate` column using the `month()` function. Boston already has this column.

In this example, we use several functions in the `pyspark.sql.functions` library, and need to import:

* `month()` to extract the month from `reportDate` timestamp data type.
* `lower()` to convert text to lowercase.
* `contains(mySubstr)` to indicate a string contains substring `mySubstr`.

Also, note we use  `|`  to indicate a logical `or` of two conditions in the `filter` method.

In [44]:
crimeDataNewYorkHomicideDF=crimeDataNewYorkDF.filter(crimeDataNewYorkDF.offenseDescription.startswith("HOMICIDE") |crimeDataNewYorkDF.offenseDescription.startswith("MURDER"))
HomicideByMonthNewYorkDF=crimeDataNewYorkHomicideDF.select(month(crimeDataNewYorkDF.reportDate).alias('MONTH'),crimeDataNewYorkHomicideDF.offenseDescription).groupBy("MONTH").count().sort(col('month').asc())
display(HomicideByMonthNewYorkDF)

MONTH,count
1,22
2,17
3,25
4,28
5,31
6,32
7,30
8,36
9,34
10,20


In [45]:
crimeDataBostonHomicideDF=crimeDataBostonDF.filter(lower(crimeDataBostonDF.OFFENSE_CODE_GROUP).contains("homicide"))
HomicideByMonthBostonDF=crimeDataBostonHomicideDF.select(crimeDataBostonHomicideDF.MONTH,crimeDataBostonHomicideDF.OFFENSE_CODE_GROUP).groupBy("MONTH").count().sort(col('MONTH').asc())
display(HomicideByMonthBostonDF)

MONTH,count
1,7
2,4
3,4
4,8
5,7
6,10
7,15
8,14
9,9
10,9


In [46]:
HomicideByMonthNewYorkDF=HomicideByMonthNewYorkDF.withColumn('City',lit('New York'))
HomicideByMonthBostonDF=HomicideByMonthBostonDF.withColumn('City',lit('Boston'))

combinedHomicideByMonthDF=HomicideByMonthNewYorkDF.union(HomicideByMonthBostonDF)
display(combinedHomicideByMonthDF)

MONTH,count,City
1,22,New York
2,17,New York
3,25,New York
4,28,New York
5,31,New York
6,32,New York
7,30,New York
8,36,New York
9,34,New York
10,20,New York


In [47]:
maxValue = combinedHomicideByMonthDF.agg({"count" : "max"}).collect()[0][0]
minValue = combinedHomicideByMonthDF.agg({"count" : "min"}).collect()[0][0]
max_homicide_month=combinedHomicideByMonthDF.where(combinedHomicideByMonthDF['count']==maxValue)
min_homicide_month=combinedHomicideByMonthDF.where(combinedHomicideByMonthDF['count']==minValue)
display(max_homicide_month)

MONTH,count,City
8,36,New York


In [48]:
display(min_homicide_month)

MONTH,count,City
2,4,Boston
3,4,Boston


-sandbox
## Question 9: Find the "per HOMICIDE robbery rates" using ther Hint in Qn 7, and plot graph as above for the per capita HOMICIDE rates per month for each of the two cities, producing one Graph using the contents of HOMICIDERatesByCityDF:

In [50]:
import pyspark.sql.functions as F
homicideRatesByCityDF = combinedHomicideByMonthDF.withColumn("HomicideRate",F.col("count") / cityDataDF.select(cityDataDF.estPopulation2016).where(cityDataDF.city == F.col("city")).collect()[0][0])
display(homicideRatesByCityDF)

MONTH,count,City,HomicideRate
1,22,New York,2.5768145488823477E-6
2,17,New York,1.9911748786818143E-6
3,25,New York,2.928198351002668E-6
4,28,New York,3.2795821531229878E-6
5,31,New York,3.630965955243308E-6
6,32,New York,3.748093889283415E-6
7,30,New York,3.5138380212032014E-6
8,36,New York,4.216605625443842E-6
9,34,New York,3.982349757363629E-6
10,20,New York,2.342558680802134E-6


-sandbox
## Question 10: A stretch goal to address a Data Science question on predicting crimes for a city as a time series model. How would you predict future values for monthly Robbery Count rate for Log Angeles using the historical values from crime-data-2016.  Data Science is a multi-year discipline.  I am not expecting anything fancy.  There is no wrong answer. I do expect students to explore and give their best shot.

In [52]:
y=crimeDataLosAngelesRobberyDF.groupby(month(crimeDataLosAngelesRobberyDF.dateOccurred).alias("month")).count()
display(y)

month,count
12,936
1,846
6,841
3,819
5,876
9,825
4,806
8,857
7,926
10,951


In [53]:
from pyspark.sql.functions import collect_list
Y = rob_la.select(collect_list(rob_la['count'])).first()[0]
X = [float(i) for i in Y]
print(X)


[958.0, 835.0, 827.0, 810.0, 888.0, 834.0, 804.0, 850.0, 931.0, 932.0, 849.0, 757.0]

In [54]:
from statsmodels.tsa.arima_model import ARIMA
from pyspark.sql.functions import collect_list
from sklearn.metrics import mean_squared_error

size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

for t in range(len(test)):
	model = ARIMA(history, order=(0,0,0))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0][0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))

predicted=850.857143, expected=850.000000
predicted=850.750000, expected=931.000000
predicted=859.666667, expected=932.000000
predicted=866.900000, expected=849.000000
predicted=865.272727, expected=757.000000

## References

The crime data used in this notebook comes from the following locations:

| City          | Original Data 
| ------------- | -------------
| Boston        | <a href="https://data.boston.gov/group/public-safety" target="_blank">https&#58;//data.boston.gov/group/public-safety</a>
| Chicago       | <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2" target="_blank">https&#58;//data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2</a>
| Dallas        | <a href="https://www.dallasopendata.com/Public-Safety/Police-Incidents/tbnj-w5hb/data" target="_blank">https&#58;//www.dallasopendata.com/Public-Safety/Police-Incidents/tbnj-w5hb/data</a>
| Los Angeles   | <a href="https://data.lacity.org/A-Safe-City/Crime-Data-From-2010-to-Present/y8tr-7khq" target="_blank">https&#58;//data.lacity.org/A-Safe-City/Crime-Data-From-2010-to-Present/y8tr-7khq</a>
| New Orleans   | <a href="https://data.nola.gov/Public-Safety-and-Preparedness/Electronic-Police-Report-2016/4gc2-25he/data" target="_blank">https&#58;//data.nola.gov/Public-Safety-and-Preparedness/Electronic-Police-Report-2016/4gc2-25he/data</a>
| New York      | <a href="https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i" target="_blank">https&#58;//data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i</a>
| Philadelphia  | <a href="https://www.opendataphilly.org/dataset/crime-incidents" target="_blank">https&#58;//www.opendataphilly.org/dataset/crime-incidents</a>